In [ ]:
!pip install torchtuples
!pip install pycox
!pip install datasets
!pip install pandas==1.5.3

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
import torch
import torchtuples as tt
import pandas as pd
from pycox.models import DeepHitSingle
from sklearn.model_selection import train_test_split
from pycox.evaluation import EvalSurv
from sklearn.model_selection import StratifiedKFold, train_test_split

In [ ]:
def Average(lst):
    return sum(lst) / len(lst)

# For Synthetic Experiment

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Datasets/aids.csv')
df=df.drop('Unnamed: 0',axis=1)
df = df[df['duration'] != 0]

X=df.drop('event',axis=1)
Y=df['raceth']

C_index=[]
Brier_score=[]

C_index_race1=[]
C_index_race2=[]
C_index_race3=[]

Brier_score_race1=[]
Brier_score_race2=[]
Brier_score_race3=[]

Final_C_index=[]
Final_C_index_race1=[]
Final_C_index_race2=[]
Final_C_index_race3=[]

Final_Brier_score=[]
Final_Brier_score_race1=[]
Final_Brier_score_race2=[]
Final_Brier_score_race3=[]

for i in range(1,6):
  # random_state=i
  skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=40)
  for train_index, test_index in skf.split(X, Y):

    df_train=pd.read_csv('Path_to_synthetic_aids_csv')
    df_train= df_train[df_train['duration'] != 0]
    df_train=df_train.loc[df_train.index[train_index]]

    df_test=df.loc[df.index[test_index]]

    df_val = df_train.sample(frac=0.2)
    df_train = df_train.drop(df_val.index)

    df_test1 = df_test[df_test["raceth"] == 1]
    df_test2 = df_test[df_test["raceth"] == 2]
    df_test3 = df_test[df_test["raceth"] == 3]

    cols_standardize = []
    cols_leave = ['age', 'cd4', 'hemophil', 'ivdrug', 'karnof', 'priorzdv','raceth', 'sex', 'strat2', 'tx', 'txgrp' ]
    standardize = [([col], StandardScaler()) for col in cols_standardize]
    leave = [(col, None) for col in cols_leave]
    x_mapper = DataFrameMapper(standardize + leave)

    x_train = x_mapper.fit_transform(df_train).astype('float32')
    x_val = x_mapper.transform(df_val).astype('float32')
    x_test = x_mapper.transform(df_test).astype('float32')

    x_test1 = x_mapper.transform(df_test1).astype('float32')
    x_test2 = x_mapper.transform(df_test2).astype('float32')
    x_test3 = x_mapper.transform(df_test3).astype('float32')

    num_durations = 1000
    labtrans = DeepHitSingle.label_transform(num_durations)
    get_target = lambda df: (df['duration'].values, df['event'].values)
    y_train = labtrans.fit_transform(*get_target(df_train))
    y_val = labtrans.transform(*get_target(df_val))

    train = (x_train, y_train)
    val = (x_val, y_val)

    durations_test, events_test = get_target(df_test)

    durations_test1, events_test1 = get_target(df_test1)
    durations_test2, events_test2 = get_target(df_test2)
    durations_test3, events_test3= get_target(df_test3)

    in_features = x_train.shape[1]
    out_features = labtrans.out_features

    net = torch.nn.Sequential(
      torch.nn.Linear(in_features, 32),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(32),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(32, 32),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(32),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(32, out_features)
    )

    model = DeepHitSingle(net, tt.optim.Adam(0.01), duration_index=labtrans.cuts)

    batch_size = 256
    epochs = 100
    callbacks = [tt.cb.EarlyStopping()]
    log = model.fit(x_train, y_train, batch_size, epochs, callbacks, val_data=val)

    surv = model.predict_surv_df(x_test)
    ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
    c=ev.concordance_td('antolini')
    C_index.append(c)
    # print(c)

    surv1 = model.predict_surv_df(x_test1)
    ev1= EvalSurv(surv1, durations_test1, events_test1, censor_surv='km')
    c1=ev1.concordance_td('antolini')
    C_index_race1.append(c1)

    surv2 = model.predict_surv_df(x_test2)
    ev2= EvalSurv(surv2, durations_test2, events_test2, censor_surv='km')
    c2=ev1.concordance_td('antolini')
    C_index_race2.append(c2)

    surv3= model.predict_surv_df(x_test3)
    ev3= EvalSurv(surv3, durations_test3, events_test3, censor_surv='km')
    c3=ev3.concordance_td('antolini')
    C_index_race3.append(c3)

    time_grid = np.linspace(durations_test.min(), durations_test.max(), 100)
    b=ev.integrated_brier_score(time_grid)
    Brier_score.append(b)

    time_grid1= np.linspace(durations_test1.min(), durations_test1.max(), 100)
    b1=ev.integrated_brier_score(time_grid1)
    Brier_score_race1.append(b1)

    time_grid2= np.linspace(durations_test2.min(), durations_test2.max(), 100)
    b2=ev.integrated_brier_score(time_grid2)
    Brier_score_race2.append(b2)

    time_grid3= np.linspace(durations_test3.min(), durations_test3.max(), 100)
    b3=ev.integrated_brier_score(time_grid3)
    Brier_score_race3.append(b3)

  average_C_index=Average(C_index)
  average_C_index_race1=Average(C_index_race1)
  average_C_index_race2=Average(C_index_race2)
  average_C_index_race3=Average(C_index_race3)

  average_Brier_score=Average(Brier_score)
  average_Brier_score_race1=Average(Brier_score_race1)
  average_Brier_score_race2=Average(Brier_score_race2)
  average_Brier_score_race3=Average(Brier_score_race3)

  Final_C_index.append(average_C_index)
  Final_C_index_race1.append(average_C_index_race1)
  Final_C_index_race2.append(average_C_index_race2)
  Final_C_index_race3.append(average_C_index_race3)

  Final_Brier_score.append(average_Brier_score)
  Final_Brier_score_race1.append(average_Brier_score_race1)
  Final_Brier_score_race2.append(average_Brier_score_race2)
  Final_Brier_score_race3.append(average_Brier_score_race3)
# print("C_index =", round(average_C_index, 3))
# print("C_index race 1 = ",round(average_C_index_race1, 3))
# print("C_index race 2 = ",round(average_C_index_race2, 3))
# print("C_index race 3 = ",round(average_C_index_race3, 3))

# print("Brier_score =", round(average_Brier_score, 3))
# print("Brier_score race 1 = ",round(average_Brier_score_race1, 3))
# print("Brier_score race 2 = ",round(average_Brier_score_race2, 3))
# print("Brier_score race 3 = ",round(average_Brier_score_race3, 3))

In [ ]:
print(Average(Final_C_index))
print(Average(Final_C_index_race1))
print(Average(Final_C_index_race2))
print(Average(Final_C_index_race3))

In [ ]:
print(Average(Final_Brier_score))
print(Average(Final_Brier_score_race1))
print(Average(Final_Brier_score_race2))
print(Average(Final_Brier_score_race3))

# For Original data

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Datasets/aids.csv')
df=df.drop('Unnamed: 0',axis=1)
df = df[df['duration'] != 0]

X=df.drop('event',axis=1)
Y=df['raceth']

C_index=[]
Brier_score=[]

C_index_race1=[]
C_index_race2=[]
C_index_race3=[]

Brier_score_race1=[]
Brier_score_race2=[]
Brier_score_race3=[]

Final_C_index=[]
Final_C_index_race1=[]
Final_C_index_race2=[]
Final_C_index_race3=[]

Final_Brier_score=[]
Final_Brier_score_race1=[]
Final_Brier_score_race2=[]
Final_Brier_score_race3=[]

for i in range(5):
  # random_state=i
  skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=40)
  for train_index, test_index in skf.split(X, Y):

    df_train=df.loc[df.index[train_index]]
    df_test=df.loc[df.index[test_index]]

    df_val = df_train.sample(frac=0.2)
    df_train = df_train.drop(df_val.index)

    # print(np.shape(df_train))
    # print(np.shape(df_test))

    df_test1 = df_test[df_test["raceth"] == 1]
    df_test2 = df_test[df_test["raceth"] == 2]
    df_test3 = df_test[df_test["raceth"] == 3]


    cols_standardize = []
    cols_leave = ['age', 'cd4', 'hemophil', 'ivdrug', 'karnof', 'priorzdv','raceth', 'sex', 'strat2', 'tx', 'txgrp' ]
    standardize = [([col], StandardScaler()) for col in cols_standardize]
    leave = [(col, None) for col in cols_leave]
    x_mapper = DataFrameMapper(standardize + leave)

    x_train = x_mapper.fit_transform(df_train).astype('float32')
    x_val = x_mapper.transform(df_val).astype('float32')
    x_test = x_mapper.transform(df_test).astype('float32')

    x_test1 = x_mapper.transform(df_test1).astype('float32')
    x_test2 = x_mapper.transform(df_test2).astype('float32')
    x_test3 = x_mapper.transform(df_test3).astype('float32')

    num_durations = 1000
    labtrans = DeepHitSingle.label_transform(num_durations)
    get_target = lambda df: (df['duration'].values, df['event'].values)
    y_train = labtrans.fit_transform(*get_target(df_train))
    y_val = labtrans.transform(*get_target(df_val))

    train = (x_train, y_train)
    val = (x_val, y_val)

    durations_test, events_test = get_target(df_test)

    durations_test1, events_test1 = get_target(df_test1)
    durations_test2, events_test2 = get_target(df_test2)
    durations_test3, events_test3= get_target(df_test3)

    in_features = x_train.shape[1]
    out_features = labtrans.out_features

    net = torch.nn.Sequential(
      torch.nn.Linear(in_features, 32),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(32),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(32, 32),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(32),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(32, out_features)
    )

    model = DeepHitSingle(net, tt.optim.Adam(0.01), duration_index=labtrans.cuts)

    batch_size = 256
    epochs = 100
    callbacks = [tt.cb.EarlyStopping()]
    log = model.fit(x_train, y_train, batch_size, epochs, callbacks, val_data=val)

    surv = model.predict_surv_df(x_test)
    ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
    c=ev.concordance_td('antolini')
    C_index.append(c)
    # print(c)

    surv1 = model.predict_surv_df(x_test1)
    ev1= EvalSurv(surv1, durations_test1, events_test1, censor_surv='km')
    c1=ev1.concordance_td('antolini')
    C_index_race1.append(c1)

    surv2 = model.predict_surv_df(x_test2)
    ev2= EvalSurv(surv2, durations_test2, events_test2, censor_surv='km')
    c2=ev1.concordance_td('antolini')
    C_index_race2.append(c2)

    surv3= model.predict_surv_df(x_test3)
    ev3= EvalSurv(surv3, durations_test3, events_test3, censor_surv='km')
    c3=ev3.concordance_td('antolini')
    C_index_race3.append(c3)

    time_grid = np.linspace(durations_test.min(), durations_test.max(), 100)
    b=ev.integrated_brier_score(time_grid)
    Brier_score.append(b)

    time_grid1= np.linspace(durations_test1.min(), durations_test1.max(), 100)
    b1=ev.integrated_brier_score(time_grid1)
    Brier_score_race1.append(b1)

    time_grid2= np.linspace(durations_test2.min(), durations_test2.max(), 100)
    b2=ev.integrated_brier_score(time_grid2)
    Brier_score_race2.append(b2)

    time_grid3= np.linspace(durations_test3.min(), durations_test3.max(), 100)
    b3=ev.integrated_brier_score(time_grid3)
    Brier_score_race3.append(b3)

  average_C_index=Average(C_index)
  average_C_index_race1=Average(C_index_race1)
  average_C_index_race2=Average(C_index_race2)
  average_C_index_race3=Average(C_index_race3)

  average_Brier_score=Average(Brier_score)
  average_Brier_score_race1=Average(Brier_score_race1)
  average_Brier_score_race2=Average(Brier_score_race2)
  average_Brier_score_race3=Average(Brier_score_race3)

  Final_C_index.append(average_C_index)
  Final_C_index_race1.append(average_C_index_race1)
  Final_C_index_race2.append(average_C_index_race2)
  Final_C_index_race3.append(average_C_index_race3)

  Final_Brier_score.append(average_Brier_score)
  Final_Brier_score_race1.append(average_Brier_score_race1)
  Final_Brier_score_race2.append(average_Brier_score_race2)
  Final_Brier_score_race3.append(average_Brier_score_race3)
# print("C_index =", round(average_C_index, 3))
# print("C_index race 1 = ",round(average_C_index_race1, 3))
# print("C_index race 2 = ",round(average_C_index_race2, 3))
# print("C_index race 3 = ",round(average_C_index_race3, 3))

# print("Brier_score =", round(average_Brier_score, 3))
# print("Brier_score race 1 = ",round(average_Brier_score_race1, 3))
# print("Brier_score race 2 = ",round(average_Brier_score_race2, 3))
# print("Brier_score race 3 = ",round(average_Brier_score_race3, 3))

In [ ]:
print(Average(Final_C_index))
print(Average(Final_C_index_race1))
print(Average(Final_C_index_race2))
print(Average(Final_C_index_race3))

In [ ]:
print(Average(Final_Brier_score))
print(Average(Final_Brier_score_race1))
print(Average(Final_Brier_score_race2))
print(Average(Final_Brier_score_race3))

# For Synthetic (Balanced)

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Datasets/aids.csv')
df=df.drop('Unnamed: 0',axis=1)
df = df[df['duration'] != 0]

X=df.drop('event',axis=1)
Y=df['raceth']

C_index=[]
Brier_score=[]

C_index_race1=[]
C_index_race2=[]
C_index_race3=[]

Brier_score_race1=[]
Brier_score_race2=[]
Brier_score_race3=[]

Final_C_index=[]
Final_C_index_race1=[]
Final_C_index_race2=[]
Final_C_index_race3=[]

Final_Brier_score=[]
Final_Brier_score_race1=[]
Final_Brier_score_race2=[]
Final_Brier_score_race3=[]

for i in range(1,6):
  # random_state=i
  skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=40)
  for train_index, test_index in skf.split(X, Y):

    df_train=pd.read_csv('path_to_synthetic_balanced_aids')
    df_train= df_train[df_train['duration'] != 0]
    # df_train=df_train.loc[df_train.index[train_index]]

    df_test=df.loc[df.index[test_index]]

    df_val = df_train.sample(frac=0.2)
    df_train = df_train.drop(df_val.index)

    df_test1 = df_test[df_test["raceth"] == 1]
    df_test2 = df_test[df_test["raceth"] == 2]
    df_test3 = df_test[df_test["raceth"] == 3]

    cols_standardize = []
    cols_leave = ['age', 'cd4', 'hemophil', 'ivdrug', 'karnof', 'priorzdv','raceth', 'sex', 'strat2', 'tx', 'txgrp' ]
    standardize = [([col], StandardScaler()) for col in cols_standardize]
    leave = [(col, None) for col in cols_leave]
    x_mapper = DataFrameMapper(standardize + leave)

    x_train = x_mapper.fit_transform(df_train).astype('float32')
    x_val = x_mapper.transform(df_val).astype('float32')
    x_test = x_mapper.transform(df_test).astype('float32')

    x_test1 = x_mapper.transform(df_test1).astype('float32')
    x_test2 = x_mapper.transform(df_test2).astype('float32')
    x_test3 = x_mapper.transform(df_test3).astype('float32')

    num_durations = 1000
    labtrans = DeepHitSingle.label_transform(num_durations)
    get_target = lambda df: (df['duration'].values, df['event'].values)
    y_train = labtrans.fit_transform(*get_target(df_train))
    y_val = labtrans.transform(*get_target(df_val))

    train = (x_train, y_train)
    val = (x_val, y_val)

    durations_test, events_test = get_target(df_test)

    durations_test1, events_test1 = get_target(df_test1)
    durations_test2, events_test2 = get_target(df_test2)
    durations_test3, events_test3= get_target(df_test3)

    in_features = x_train.shape[1]
    out_features = labtrans.out_features

    net = torch.nn.Sequential(
      torch.nn.Linear(in_features, 32),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(32),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(32, 32),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(32),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(32, out_features)
    )

    model = DeepHitSingle(net, tt.optim.Adam(0.01), duration_index=labtrans.cuts)

    batch_size = 256
    epochs = 100
    callbacks = [tt.cb.EarlyStopping()]
    log = model.fit(x_train, y_train, batch_size, epochs, callbacks, val_data=val)

    surv = model.predict_surv_df(x_test)
    ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
    c=ev.concordance_td('antolini')
    C_index.append(c)
    # print(c)

    surv1 = model.predict_surv_df(x_test1)
    ev1= EvalSurv(surv1, durations_test1, events_test1, censor_surv='km')
    c1=ev1.concordance_td('antolini')
    C_index_race1.append(c1)

    surv2 = model.predict_surv_df(x_test2)
    ev2= EvalSurv(surv2, durations_test2, events_test2, censor_surv='km')
    c2=ev1.concordance_td('antolini')
    C_index_race2.append(c2)

    surv3= model.predict_surv_df(x_test3)
    ev3= EvalSurv(surv3, durations_test3, events_test3, censor_surv='km')
    c3=ev3.concordance_td('antolini')
    C_index_race3.append(c3)

    time_grid = np.linspace(durations_test.min(), durations_test.max(), 100)
    b=ev.integrated_brier_score(time_grid)
    Brier_score.append(b)

    time_grid1= np.linspace(durations_test1.min(), durations_test1.max(), 100)
    b1=ev.integrated_brier_score(time_grid1)
    Brier_score_race1.append(b1)

    time_grid2= np.linspace(durations_test2.min(), durations_test2.max(), 100)
    b2=ev.integrated_brier_score(time_grid2)
    Brier_score_race2.append(b2)

    time_grid3= np.linspace(durations_test3.min(), durations_test3.max(), 100)
    b3=ev.integrated_brier_score(time_grid3)
    Brier_score_race3.append(b3)

  average_C_index=Average(C_index)
  average_C_index_race1=Average(C_index_race1)
  average_C_index_race2=Average(C_index_race2)
  average_C_index_race3=Average(C_index_race3)

  average_Brier_score=Average(Brier_score)
  average_Brier_score_race1=Average(Brier_score_race1)
  average_Brier_score_race2=Average(Brier_score_race2)
  average_Brier_score_race3=Average(Brier_score_race3)

  Final_C_index.append(average_C_index)
  Final_C_index_race1.append(average_C_index_race1)
  Final_C_index_race2.append(average_C_index_race2)
  Final_C_index_race3.append(average_C_index_race3)

  Final_Brier_score.append(average_Brier_score)
  Final_Brier_score_race1.append(average_Brier_score_race1)
  Final_Brier_score_race2.append(average_Brier_score_race2)
  Final_Brier_score_race3.append(average_Brier_score_race3)
# print("C_index =", round(average_C_index, 3))
# print("C_index race 1 = ",round(average_C_index_race1, 3))
# print("C_index race 2 = ",round(average_C_index_race2, 3))
# print("C_index race 3 = ",round(average_C_index_race3, 3))

# print("Brier_score =", round(average_Brier_score, 3))
# print("Brier_score race 1 = ",round(average_Brier_score_race1, 3))
# print("Brier_score race 2 = ",round(average_Brier_score_race2, 3))
# print("Brier_score race 3 = ",round(average_Brier_score_race3, 3))

In [ ]:
print(Average(Final_C_index))
print(Average(Final_C_index_race1))
print(Average(Final_C_index_race2))
print(Average(Final_C_index_race3))

In [ ]:
print(Average(Final_Brier_score))
print(Average(Final_Brier_score_race1))
print(Average(Final_Brier_score_race2))
print(Average(Final_Brier_score_race3))